In [119]:
import pandas as pd
df = pd.read_csv('D:\Embedding\Code\Patients.csv')
df.head()

,SUBJECT_ID,HADM_ID
0,68591,100016
1,29633,100050
2,14121,100058
3,26885,100074
4,24882,100109


In [120]:
len(df)

4732

In [3]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="azkaban123456",
  database="mimiciiiv14"
)


In [8]:
txt_counter=0

mycursor = mydb.cursor()
str_PHX="SELECT mn.SUBJECT_ID,mn.HADM_ID,text  FROM mimiciiiv14.noteevents mn \
                inner join project_patient pp on pp.SUBJECT_ID=mn.SUBJECT_ID and pp.HADM_ID=mn.HADM_ID\
                where mn.CATEGORY like 'Discharge summary' and mn.DESCRIPTION like 'report'  "

print(str_PHX)
mycursor.execute(str_PHX)
num_fields = len(mycursor.description)
field_names = [i[0] for i in mycursor.description]

myresult = mycursor.fetchall()
    


SELECT mn.SUBJECT_ID,mn.HADM_ID,text  FROM mimiciiiv14.noteevents mn                 inner join project_patient pp on pp.SUBJECT_ID=mn.SUBJECT_ID and pp.HADM_ID=mn.HADM_ID                where mn.CATEGORY like 'Discharge summary' and mn.DESCRIPTION like 'report'  


In [9]:
df_phx=pd.DataFrame(myresult,columns=['SUBJECT_ID','HADM_ID','TEXT'])

In [10]:
df_phx.to_csv('D:\Embedding\Code\Data\phx.csv', index = False)

In [11]:
from heuristic_tokenize import sent_tokenize_rules
def process_note_helper(note):
    # split note into sections
    
    note_sections = sent_tokenize_rules(note)
    section_frame = pd.DataFrame({'sections':note_sections})
    return section_frame

In [12]:
df_phx.iloc[93]

SUBJECT_ID                                                 4053
HADM_ID                                                  102108
TEXT          Admission Date:  [**2164-1-24**]              ...
Name: 93, dtype: object

In [13]:
import sys
import nltk
import re
import os
def sent_tokenize_rules(text):

    # long sections are OBVIOUSLY different sentences
    text = re.sub('---+', '\n\n-----\n\n', text)
    text = re.sub('___+', '\n\n_____\n\n', text)
    text = re.sub('\n\n+', '\n\n', text)
    text=text.lower()
    segments = text.split('\n\n')  
        ### Separate section headers ###
    new_segments = []

    # deal with this one edge case (multiple headers per line) up front
    m1 = re.match('(Admission Date:) (.*) (Discharge Date:) (.*)', segments[0])
    if m1:
        new_segments += list(map(str.strip,m1.groups()))
        segments = segments[1:]

    m2 = re.match('(Date of Birth:) (.*) (Sex:) (.*)'            , segments[0])
    if m2:
        new_segments += list(map(str.strip,m2.groups()))
        segments = segments[1:]

    for segment in segments:
        # find all section headers
        possible_headers  = re.findall('\n([A-Z][^\n:]+:)', '\n'+segment)
        #assert len(possible_headers) < 2, str(possible_headers)
        headers = []
        
        # split text into new segments, delimiting on these headers
        for h in headers:
            h = h.strip()

            # split this segment into 3 smaller segments
            ind = segment.index(h)
            prefix = segment[:ind].strip()
            rest   = segment[ ind+len(h):].strip()

            # add the prefix (potentially empty)
            if len(prefix) > 0:
                new_segments.append(prefix.strip())

            # add the header
            new_segments.append(h)

            # remove the prefix from processing (very unlikely to be empty)
            segment = rest.strip()

        # add the final piece (aka what comes after all headers are processed)
        if len(segment) > 0:
            new_segments.append(segment.strip())

    # copy over the new list of segments (further segmented than original segments)
    segments = list(new_segments)
    return segments

In [15]:
def Assign_Section(A):
    Dict_Section = {1: 'sex'
               , 2: 'service'
               , 3: 'chief complaint'
               , 4: 'history of present illness'
               , 5: 'past medical history'
               , 6: 'allergies'
               , 7: 'medication in admission '
               , 8: 'social history'
               , 9: 'physical exam'
               , 10: 'brief hospital course'
               , 11: 'discharge condition'
               , 12: 'discharge placement'
               , 13: 'discharge diagnosis'
               , 14: 'discharge medications'
               , 15: 'followup'
               , 16: 'dictated by'
               , 17: 'medications on admission'
               , 18: 'discharge instructions'
               , 19: 'family history'
               , 20: 'discharge disposition'  
               , 21: 'pertinent results'     
               } 
    ls_section=[]
   
    for i in range(len(A)):
        text=A[i]
        
        for j in range(1,21): 
            if(text.startswith(Dict_Section[j])) :
                current_section=Dict_Section[j]
                text = text.replace(Dict_Section[j], '').replace(':', '').replace('.', '')
                text=text.strip()
                ls_section.append((current_section,text))
                Not_Found=0
                break
            else:
                Not_Found=1

        if Not_Found==1:
            ls_section.append(('NOthing',text))      
        else:
            continue
    df = pd.DataFrame(ls_section,columns=['Section','Text'])    
    return df

In [16]:
processed_sections=sent_tokenize_rules(df_phx.iloc[2].TEXT)
df_sec=Assign_Section(processed_sections)
df_sec

,Section,Text
0,NOthing,admission date: [**2139-9-8**] discharge ...
1,NOthing,date of birth: [**2082-11-16**] sex: f
2,service,newurosurgery
3,history of present illness,the patient is a 57 year old\nfemale with no p...
4,past medical history,none
5,NOthing,past surgical history: none.
6,allergies,no known drug
7,medications on admission,none
8,social history,etoh and was a thirty pack year smoker
9,physical exam,ination neurologically she was completely\nin...


In [17]:
ls_phx_total=[]
for i in range(len(df_phx)):
    print(len(df_phx)-i)
    
    note_text = df_phx.iloc[i].TEXT
    processed_sections = sent_tokenize_rules(note_text)
    df_sec=Assign_Section(processed_sections)
    df_phx_i=df_sec[df_sec['Section']=='past medical history']
    if len(df_phx_i)>0:
        ls_text=[]
    for j in range(len (df_phx_i)):
        ls_text.append(df_phx_i.iloc[j]['Text'])
    ls_phx_total.append([df_phx.iloc[i].SUBJECT_ID,df_phx.iloc[i].HADM_ID,ls_text[0]]) 

pd.DataFrame(ls_phx_total,columns=['SUBJECT_ID','HADM_ID','PHX_TEXT'])        

4750
4749
4748
4747
4746
4745
4744
4743
4742
4741
4740
4739
4738
4737
4736
4735
4734
4733
4732
4731
4730
4729
4728
4727
4726
4725
4724
4723
4722
4721
4720
4719
4718
4717
4716
4715
4714
4713
4712
4711
4710
4709
4708
4707
4706
4705
4704
4703
4702
4701
4700
4699
4698
4697
4696
4695
4694
4693
4692
4691
4690
4689
4688
4687
4686
4685
4684
4683
4682
4681
4680
4679
4678
4677
4676
4675
4674
4673
4672
4671
4670
4669
4668
4667
4666
4665
4664
4663
4662
4661
4660
4659
4658
4657
4656
4655
4654
4653
4652
4651
4650
4649
4648
4647
4646
4645
4644
4643
4642
4641
4640
4639
4638
4637
4636
4635
4634
4633
4632
4631
4630
4629
4628
4627
4626
4625
4624
4623
4622
4621
4620
4619
4618
4617
4616
4615
4614
4613
4612
4611
4610
4609
4608
4607
4606
4605
4604
4603
4602
4601
4600
4599
4598
4597
4596
4595
4594
4593
4592
4591
4590
4589
4588
4587
4586
4585
4584
4583
4582
4581
4580
4579
4578
4577
4576
4575
4574
4573
4572
4571
4570
4569
4568
4567
4566
4565
4564
4563
4562
4561
4560
4559
4558
4557
4556
4555
4554
4553
4552
4551


3010
3009
3008
3007
3006
3005
3004
3003
3002
3001
3000
2999
2998
2997
2996
2995
2994
2993
2992
2991
2990
2989
2988
2987
2986
2985
2984
2983
2982
2981
2980
2979
2978
2977
2976
2975
2974
2973
2972
2971
2970
2969
2968
2967
2966
2965
2964
2963
2962
2961
2960
2959
2958
2957
2956
2955
2954
2953
2952
2951
2950
2949
2948
2947
2946
2945
2944
2943
2942
2941
2940
2939
2938
2937
2936
2935
2934
2933
2932
2931
2930
2929
2928
2927
2926
2925
2924
2923
2922
2921
2920
2919
2918
2917
2916
2915
2914
2913
2912
2911
2910
2909
2908
2907
2906
2905
2904
2903
2902
2901
2900
2899
2898
2897
2896
2895
2894
2893
2892
2891
2890
2889
2888
2887
2886
2885
2884
2883
2882
2881
2880
2879
2878
2877
2876
2875
2874
2873
2872
2871
2870
2869
2868
2867
2866
2865
2864
2863
2862
2861
2860
2859
2858
2857
2856
2855
2854
2853
2852
2851
2850
2849
2848
2847
2846
2845
2844
2843
2842
2841
2840
2839
2838
2837
2836
2835
2834
2833
2832
2831
2830
2829
2828
2827
2826
2825
2824
2823
2822
2821
2820
2819
2818
2817
2816
2815
2814
2813
2812
2811


1316
1315
1314
1313
1312
1311
1310
1309
1308
1307
1306
1305
1304
1303
1302
1301
1300
1299
1298
1297
1296
1295
1294
1293
1292
1291
1290
1289
1288
1287
1286
1285
1284
1283
1282
1281
1280
1279
1278
1277
1276
1275
1274
1273
1272
1271
1270
1269
1268
1267
1266
1265
1264
1263
1262
1261
1260
1259
1258
1257
1256
1255
1254
1253
1252
1251
1250
1249
1248
1247
1246
1245
1244
1243
1242
1241
1240
1239
1238
1237
1236
1235
1234
1233
1232
1231
1230
1229
1228
1227
1226
1225
1224
1223
1222
1221
1220
1219
1218
1217
1216
1215
1214
1213
1212
1211
1210
1209
1208
1207
1206
1205
1204
1203
1202
1201
1200
1199
1198
1197
1196
1195
1194
1193
1192
1191
1190
1189
1188
1187
1186
1185
1184
1183
1182
1181
1180
1179
1178
1177
1176
1175
1174
1173
1172
1171
1170
1169
1168
1167
1166
1165
1164
1163
1162
1161
1160
1159
1158
1157
1156
1155
1154
1153
1152
1151
1150
1149
1148
1147
1146
1145
1144
1143
1142
1141
1140
1139
1138
1137
1136
1135
1134
1133
1132
1131
1130
1129
1128
1127
1126
1125
1124
1123
1122
1121
1120
1119
1118
1117


,SUBJECT_ID,HADM_ID,PHX_TEXT
0,68591,100016,- downs syndrome\n- mrsa pneumonia complicated...
1,29633,100050,"aortic stenosis, coronary artery disease, gast..."
2,14121,100058,none
3,26885,100074,"s/p mvc ('[**61**]), s/p r aka, ventral hernia..."
4,24882,100109,1 cirrhosis\n2 status post esophageal varice...
...,...,...,...
4745,20949,199894,complete heart block(ppm)\npostop dvt in lue [...
4746,18239,199907,1 aortic stenosis\n2 asthmatic bronchitis for ...
4747,18239,199907,1 aortic stenosis\n2 asthmatic bronchitis for ...
4748,80825,199925,"- ischemic stroke, [**10/2139**]\n- hypertensi..."


In [18]:
df_phx_filtered=pd.DataFrame(ls_phx_total,columns=['SUBJECT_ID','HADM_ID','PHX_TEXT'])  

In [19]:
len(df_phx_filtered)

4750

In [20]:
df_phx_filtered.head()

,SUBJECT_ID,HADM_ID,PHX_TEXT
0,68591,100016,- downs syndrome\n- mrsa pneumonia complicated...
1,29633,100050,"aortic stenosis, coronary artery disease, gast..."
2,14121,100058,none
3,26885,100074,"s/p mvc ('[**61**]), s/p r aka, ventral hernia..."
4,24882,100109,1 cirrhosis\n2 status post esophageal varice...


 Create text file from PHX

In [22]:
txt_counter=0
ls_Metadata=[]
print(len(df_phx_filtered))
for j in range(len(df_phx_filtered)):
        L=df_phx_filtered.iloc[j].PHX_TEXT
        f_adr="D:\\Embedding\\Code\\Data\\Phx_Input\\" + str(txt_counter)+".txt"
        #file1 = open(f_adr,"w") 
        #file1.write(L) 
        #file1.writelines(L) 
        #file1.close() #to change file access modes 
        ls_Metadata.append([df_phx_filtered.iloc[j].SUBJECT_ID,df_phx_filtered.iloc[j].HADM_ID,txt_counter])
        txt_counter=txt_counter+1
        print(len(df_phx_filtered)- txt_counter)

4750
4749
4748
4747
4746
4745
4744
4743
4742
4741
4740
4739
4738
4737
4736
4735
4734
4733
4732
4731
4730
4729
4728
4727
4726
4725
4724
4723
4722
4721
4720
4719
4718
4717
4716
4715
4714
4713
4712
4711
4710
4709
4708
4707
4706
4705
4704
4703
4702
4701
4700
4699
4698
4697
4696
4695
4694
4693
4692
4691
4690
4689
4688
4687
4686
4685
4684
4683
4682
4681
4680
4679
4678
4677
4676
4675
4674
4673
4672
4671
4670
4669
4668
4667
4666
4665
4664
4663
4662
4661
4660
4659
4658
4657
4656
4655
4654
4653
4652
4651
4650
4649
4648
4647
4646
4645
4644
4643
4642
4641
4640
4639
4638
4637
4636
4635
4634
4633
4632
4631
4630
4629
4628
4627
4626
4625
4624
4623
4622
4621
4620
4619
4618
4617
4616
4615
4614
4613
4612
4611
4610
4609
4608
4607
4606
4605
4604
4603
4602
4601
4600
4599
4598
4597
4596
4595
4594
4593
4592
4591
4590
4589
4588
4587
4586
4585
4584
4583
4582
4581
4580
4579
4578
4577
4576
4575
4574
4573
4572
4571
4570
4569
4568
4567
4566
4565
4564
4563
4562
4561
4560
4559
4558
4557
4556
4555
4554
4553
4552
4551


3040
3039
3038
3037
3036
3035
3034
3033
3032
3031
3030
3029
3028
3027
3026
3025
3024
3023
3022
3021
3020
3019
3018
3017
3016
3015
3014
3013
3012
3011
3010
3009
3008
3007
3006
3005
3004
3003
3002
3001
3000
2999
2998
2997
2996
2995
2994
2993
2992
2991
2990
2989
2988
2987
2986
2985
2984
2983
2982
2981
2980
2979
2978
2977
2976
2975
2974
2973
2972
2971
2970
2969
2968
2967
2966
2965
2964
2963
2962
2961
2960
2959
2958
2957
2956
2955
2954
2953
2952
2951
2950
2949
2948
2947
2946
2945
2944
2943
2942
2941
2940
2939
2938
2937
2936
2935
2934
2933
2932
2931
2930
2929
2928
2927
2926
2925
2924
2923
2922
2921
2920
2919
2918
2917
2916
2915
2914
2913
2912
2911
2910
2909
2908
2907
2906
2905
2904
2903
2902
2901
2900
2899
2898
2897
2896
2895
2894
2893
2892
2891
2890
2889
2888
2887
2886
2885
2884
2883
2882
2881
2880
2879
2878
2877
2876
2875
2874
2873
2872
2871
2870
2869
2868
2867
2866
2865
2864
2863
2862
2861
2860
2859
2858
2857
2856
2855
2854
2853
2852
2851
2850
2849
2848
2847
2846
2845
2844
2843
2842
2841


1355
1354
1353
1352
1351
1350
1349
1348
1347
1346
1345
1344
1343
1342
1341
1340
1339
1338
1337
1336
1335
1334
1333
1332
1331
1330
1329
1328
1327
1326
1325
1324
1323
1322
1321
1320
1319
1318
1317
1316
1315
1314
1313
1312
1311
1310
1309
1308
1307
1306
1305
1304
1303
1302
1301
1300
1299
1298
1297
1296
1295
1294
1293
1292
1291
1290
1289
1288
1287
1286
1285
1284
1283
1282
1281
1280
1279
1278
1277
1276
1275
1274
1273
1272
1271
1270
1269
1268
1267
1266
1265
1264
1263
1262
1261
1260
1259
1258
1257
1256
1255
1254
1253
1252
1251
1250
1249
1248
1247
1246
1245
1244
1243
1242
1241
1240
1239
1238
1237
1236
1235
1234
1233
1232
1231
1230
1229
1228
1227
1226
1225
1224
1223
1222
1221
1220
1219
1218
1217
1216
1215
1214
1213
1212
1211
1210
1209
1208
1207
1206
1205
1204
1203
1202
1201
1200
1199
1198
1197
1196
1195
1194
1193
1192
1191
1190
1189
1188
1187
1186
1185
1184
1183
1182
1181
1180
1179
1178
1177
1176
1175
1174
1173
1172
1171
1170
1169
1168
1167
1166
1165
1164
1163
1162
1161
1160
1159
1158
1157
1156


In [23]:
ls_Metadata

[[68591, 100016, 0],
 [29633, 100050, 1],
 [14121, 100058, 2],
 [26885, 100074, 3],
 [24882, 100109, 4],
 [1601, 100160, 5],
 [6741, 100191, 6],
 [16550, 100209, 7],
 [81865, 100215, 8],
 [52779, 100227, 9],
 [15254, 100229, 10],
 [54120, 100263, 11],
 [67117, 100325, 12],
 [21019, 100342, 13],
 [12049, 100364, 14],
 [93742, 100383, 15],
 [9141, 100403, 16],
 [63031, 100416, 17],
 [80030, 100442, 18],
 [54077, 100454, 19],
 [91929, 100463, 20],
 [9120, 100510, 21],
 [12508, 100524, 22],
 [20318, 100638, 23],
 [22463, 100639, 24],
 [631, 100660, 25],
 [76191, 100740, 26],
 [88106, 100746, 27],
 [720, 100753, 28],
 [10822, 100814, 29],
 [15303, 100837, 30],
 [27710, 100852, 31],
 [26067, 100859, 32],
 [40851, 100877, 33],
 [20643, 100883, 34],
 [6767, 100950, 35],
 [28123, 100983, 36],
 [29102, 100995, 37],
 [15153, 100998, 38],
 [32367, 101033, 39],
 [32013, 101043, 40],
 [16301, 101057, 41],
 [31325, 101098, 42],
 [12439, 101106, 43],
 [8962, 101126, 44],
 [23734, 101143, 45],
 [24089,

cTakes for parsing xmi file

In [24]:
import os
import pandas as pd
import pytest
from ctakes_parser import ctakes_parser as parser

In [26]:
xmi_df = parser.parse_file(file_path='D:\\Embedding\\Code\\Data\\Phx_Output\\0.txt.xmi')
vv_SUBJECT_ID=ls_Metadata[0][0] 
vv_HADM_ID=ls_Metadata[0][1] 
vv_txt_counter=ls_Metadata[0][2] 

xmi_df['SUBJECT_ID'] = vv_SUBJECT_ID
xmi_df['HADM_ID'] = vv_HADM_ID  
xmi_df['txt_counter'] = vv_txt_counter 
xmi_df.drop_duplicates(subset ="cui", keep='last', inplace = True)
xmi_df

,conditional,confidence,cui,generic,id,negated,pos_end,pos_start,preferred_text,refsem,...,score,subject,textsem,tui,uncertainty,true_text,part_of_speech,SUBJECT_ID,HADM_ID,txt_counter
14,True,0.0,C0701042,True,3692,False,680,675,Cipro,UmlsConcept,...,0.0,patient,MedicationMention,T109,0.0,cipro,NN,68591,100016,0
23,True,0.0,C0014563,True,3870,False,707,700,Epinephrine,UmlsConcept,...,0.0,patient,MedicationMention,T121,0.0,adrenal,JJ,68591,100016,0
25,True,0.0,C0876060,True,3756,False,828,822,Keppra,UmlsConcept,...,0.0,patient,MedicationMention,T121,0.0,keppra,NN,68591,100016,0
39,True,0.0,C0013080,True,4591,False,491,477,Down Syndrome,UmlsConcept,...,0.0,patient,DiseaseDisorderMention,T047,0.0,downs syndrome,NNS,68591,100016,0
40,True,0.0,C0039082,True,4899,False,491,483,Syndrome,UmlsConcept,...,0.0,patient,DiseaseDisorderMention,T047,0.0,syndrome,NN,68591,100016,0
41,True,0.0,C1142536,True,4437,False,509,495,Pneumonia due to methicillin resistant Staphyl...,UmlsConcept,...,0.0,patient,DiseaseDisorderMention,T047,0.0,mrsa pneumonia,NN,68591,100016,0
42,True,0.0,C0343401,True,3931,False,499,495,MRSA - Methicillin resistant Staphylococcus au...,UmlsConcept,...,0.0,patient,DiseaseDisorderMention,T047,0.0,mrsa,NN,68591,100016,0
43,True,0.0,C0032285,True,4855,False,509,500,Pneumonia,UmlsConcept,...,0.0,patient,DiseaseDisorderMention,T047,0.0,pneumonia,NN,68591,100016,0
45,True,0.0,C0009319,True,4151,False,635,628,Colitis,UmlsConcept,...,0.0,patient,DiseaseDisorderMention,T047,0.0,colitis,NN,68591,100016,0
48,True,0.0,C0001623,True,4381,False,721,700,Adrenal gland hypofunction,UmlsConcept,...,0.0,patient,DiseaseDisorderMention,T047,0.0,adrenal insufficiency,JJ,68591,100016,0


In [27]:
vv_SUBJECT_ID=[]
vv_HADM_ID=[]
vv_CHARTDATE=[]
vv_txt_counter=[]

repository_df = pd.DataFrame(columns=['cui','pos_start','negated','preferred_text','textsem','true_text','part_of_speech','SUBJECT_ID','HADM_ID','CHARTDATE','txt_counter'])
for i in range(len(ls_Metadata)):
    print(len(ls_Metadata)-i)
    xmi_df = pd.DataFrame(columns=['cui','pos_start','negated','preferred_text','textsem','true_text','part_of_speech','SUBJECT_ID','HADM_ID','CHARTDATE','txt_counter'])
    try:
        xmi_df = parser.parse_file(file_path='D:\\Embedding\\Code\\Data\\Phx_Output\\'+str(i)+'.txt.xmi')
        xmi_df.sort_values('pos_start')
        xmi_df=xmi_df[['cui','pos_start','negated','preferred_text','textsem','true_text','part_of_speech']]
        
        #xmi_df=xmi_df.loc[(xmi_df['true_text']!='Plan') & (xmi_df['true_text']!='Nursing') & (xmi_df['textsem']!='ProcedureMention')& (xmi_df['textsem']!='MedicationMention') ].drop_duplicates().sort_values('pos_start')
        #xmi_df=xmi_df.loc[(xmi_df['true_text']!=('plan','Plan','PLAN'))& (xmi_df['true_text']!='Nursing') & (xmi_df['textsem']=='DiseaseDisorderMention')]
        vv_SUBJECT_ID=ls_Metadata[i][0] 
        vv_HADM_ID=ls_Metadata[i][1] 
        vv_txt_counter=ls_Metadata[i][2] 

        xmi_df['SUBJECT_ID'] = vv_SUBJECT_ID
        xmi_df['HADM_ID'] = vv_HADM_ID  
        xmi_df['txt_counter'] = vv_txt_counter 
        xmi_df.drop_duplicates(subset ="cui", keep='last', inplace = True)
    except:
        print("An exception occurred")

    
    if len(xmi_df)>0:
        frames = [repository_df,xmi_df]
        repository_df = pd.concat(frames)
        repository_df.reset_index(drop=True)    

4750
4749
4748
An exception occurred
4747
4746
4745
4744
4743
4742
4741
4740
4739
4738
4737
4736
4735
4734
4733
4732
4731
4730
4729
An exception occurred
4728
4727
4726
4725
4724
4723
4722
4721
4720
An exception occurred
4719
4718
4717
4716
4715
4714
4713
4712
4711
4710
4709
4708
4707
4706
4705
4704
4703
4702
4701
4700
An exception occurred
4699
4698
4697
4696
4695
4694
4693
4692
4691
4690
4689
4688
4687
4686
4685
4684
4683
4682
4681
4680
4679
4678
4677
4676
An exception occurred
4675
4674
4673
4672
4671
4670
4669
4668
4667
4666
An exception occurred
4665
4664
4663
4662
4661
4660
4659
4658
4657
4656
4655
4654
4653
4652
4651
4650
4649
4648
4647
4646
4645
4644
4643
4642
4641
4640
4639
4638
4637
4636
An exception occurred
4635
4634
An exception occurred
4633
4632
4631
4630
4629
4628
4627
4626
4625
4624
4623
4622
4621
4620
4619
4618
4617
4616
An exception occurred
4615
4614
4613
4612
4611
4610
An exception occurred
4609
4608
4607
4606
4605
4604
4603
4602
4601
4600
4599
4598
4597
4596
4595


3357
3356
3355
3354
3353
3352
3351
3350
3349
3348
3347
3346
3345
3344
3343
3342
3341
3340
3339
3338
3337
3336
3335
3334
3333
3332
3331
3330
3329
3328
3327
3326
3325
3324
3323
3322
3321
3320
3319
3318
3317
3316
3315
3314
3313
3312
An exception occurred
3311
3310
3309
3308
3307
3306
3305
3304
3303
3302
3301
3300
3299
3298
3297
3296
An exception occurred
3295
3294
3293
3292
3291
3290
3289
3288
3287
3286
3285
3284
3283
3282
3281
3280
3279
3278
An exception occurred
3277
3276
3275
3274
3273
3272
3271
3270
3269
An exception occurred
3268
3267
3266
3265
3264
3263
3262
3261
3260
3259
3258
3257
3256
3255
3254
3253
3252
3251
3250
3249
3248
3247
3246
3245
3244
3243
3242
3241
3240
3239
3238
3237
3236
3235
3234
3233
3232
3231
3230
3229
3228
3227
3226
3225
3224
An exception occurred
3223
3222
3221
3220
3219
3218
3217
3216
3215
3214
3213
3212
3211
3210
3209
3208
3207
3206
3205
3204
3203
3202
3201
3200
3199
3198
3197
3196
3195
3194
3193
3192
3191
3190
3189
3188
An exception occurred
3187
3186
3185
318

1978
An exception occurred
1977
1976
1975
1974
1973
An exception occurred
1972
An exception occurred
1971
1970
1969
1968
1967
1966
1965
1964
1963
1962
1961
1960
1959
1958
1957
1956
1955
1954
1953
1952
1951
1950
1949
An exception occurred
1948
1947
1946
1945
1944
1943
1942
1941
1940
1939
1938
1937
1936
1935
1934
1933
1932
1931
1930
1929
1928
1927
1926
1925
1924
1923
1922
1921
1920
1919
An exception occurred
1918
1917
1916
1915
1914
1913
1912
1911
1910
1909
1908
1907
1906
1905
1904
1903
1902
1901
1900
1899
1898
1897
1896
1895
1894
1893
1892
1891
1890
1889
1888
1887
1886
1885
1884
1883
1882
1881
1880
1879
1878
1877
1876
1875
1874
1873
1872
1871
1870
1869
1868
1867
1866
1865
1864
1863
1862
1861
1860
1859
1858
1857
1856
1855
1854
An exception occurred
1853
1852
1851
1850
An exception occurred
1849
1848
1847
1846
1845
1844
1843
1842
1841
1840
1839
An exception occurred
1838
1837
1836
1835
1834
1833
1832
1831
1830
1829
1828
1827
1826
1825
1824
1823
1822
1821
1820
1819
1818
1817
1816
An except

499
498
497
496
495
494
493
492
491
490
489
488
487
486
485
484
483
482
481
480
479
478
477
476
475
474
473
472
471
470
469
468
467
466
465
464
463
462
461
460
459
458
457
456
455
454
453
452
451
450
449
448
447
An exception occurred
446
445
444
443
442
441
440
439
438
437
436
435
434
433
432
431
430
429
428
427
426
425
424
423
422
421
420
An exception occurred
419
418
417
An exception occurred
416
415
414
413
412
411
410
409
408
407
406
405
404
403
402
401
400
399
An exception occurred
398
An exception occurred
397
396
395
394
An exception occurred
393
392
391
390
389
388
387
386
385
384
383
382
381
380
379
378
377
376
375
374
373
372
371
370
369
368
367
366
365
364
363
362
361
360
359
358
357
356
355
354
An exception occurred
353
352
351
350
349
348
347
346
345
344
343
342
341
340
339
338
337
336
335
334
333
332
331
330
329
328
327
326
325
324
323
322
321
An exception occurred
320
319
318
317
316
315
314
313
312
311
310
309
308
307
306
305
304
303
302
301
300
299
298
297
296
295
294


negated_detection is for concatanate negation to prefered text

In [28]:
def negated_detection(a,b):
    result = b
    if a==True:
        result='not '+str(b)
    return result 

Create Stage for notes

In [29]:
Stage_df = pd.DataFrame(columns=['Subject_id','Subgraph_id','admission_Id','ago','Timestame_id','Event type','entity','value'])
Stage_df.Subject_id=repository_df.SUBJECT_ID
Stage_df.Subgraph_id=2
Stage_df.admission_Id=repository_df.HADM_ID
Stage_df.ago=repository_df.SUBJECT_ID
#Stage_df.Timestame_id=repository_df['CHARTDATE'].astype(str)
Stage_df['Event type']='DischargeSheet'
Stage_df.entity=repository_df.textsem
Stage_df.value=repository_df.apply(lambda x: negated_detection(x['negated'], x['preferred_text']), axis=1)
Stage_df=Stage_df.reset_index()

In [30]:
Stage_df

,index,Subject_id,Subgraph_id,admission_Id,ago,Timestame_id,Event type,entity,value
0,14,68591,2,100016,68591,NaN,DischargeSheet,MedicationMention,Cipro
1,23,68591,2,100016,68591,NaN,DischargeSheet,MedicationMention,Epinephrine
2,25,68591,2,100016,68591,NaN,DischargeSheet,MedicationMention,Keppra
3,39,68591,2,100016,68591,NaN,DischargeSheet,DiseaseDisorderMention,Down Syndrome
4,40,68591,2,100016,68591,NaN,DischargeSheet,DiseaseDisorderMention,Syndrome
...,...,...,...,...,...,...,...,...,...
84875,38,41115,2,199979,41115,NaN,DischargeSheet,ProcedureMention,Eye laser surgery
84876,39,41115,2,199979,41115,NaN,DischargeSheet,ProcedureMention,Ophthalmologic Surgical Procedures
84877,49,41115,2,199979,41115,NaN,DischargeSheet,AnatomicalSiteMention,Aortic valve structure
84878,51,41115,2,199979,41115,NaN,DischargeSheet,AnatomicalSiteMention,Aorta
